In [ ]:
pip install deepface opencv-python

In [25]:
import cv2
import time
import os
import sqlite3
import pickle
from deepface import DeepFace
import numpy as np

In [26]:
def create_tables():
    try:
        with sqlite3.connect('emp.db') as conn:
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS Users (
                    Id INTEGER PRIMARY KEY AUTOINCREMENT,
                    Name TEXT NOT NULL,
                    FaceEncoding BLOB
                )
            ''')
            # Create attendance table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS Attendance (
                    Id INTEGER PRIMARY KEY AUTOINCREMENT,
                    UserId INTEGER,
                    Timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY(UserId) REFERENCES Users(Id)
                )
            ''')
            print("Tables created or already exist.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

create_tables()


Tables created or already exist.


In [27]:
def detect_and_crop_faces(frames):
    face_images = []
    for frame in frames:
        try:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=True)
            if result:
                face_data = result[0]['region']
                x, y, w, h = face_data['x'], face_data['y'], face_data['w'], face_data['h']
                face_image = frame[y:y+h, x:x+w]
                face_images.append(face_image)
        except Exception as e:
            print(f"An error occurred during face detection: {e}")
    
    return face_images

In [28]:
def calculate_mean_encoding(face_images):
    encodings = []
    for face_image in face_images:
        try:
            encoding = DeepFace.represent(face_image, model_name='VGG-Face', enforce_detection=False)[0]
            encodings.append(encoding)
        except Exception as e:
            print(f"An error occurred during encoding: {e}")
    
    if encodings:
        mean_encoding = np.mean(encodings, axis=0)
        return mean_encoding
    return None



In [29]:
def capture_images(name, num_images=5, interval=1):
    cap = cv2.VideoCapture(0)
    frames = []
    
    print(f"Preparing to capture {num_images} images. Please look at the camera.")
    
    for i in range(num_images):
        time.sleep(interval)
        ret, frame = cap.read()
        
        if not ret:
            print(f"Failed to capture image {i+1}.")
            cap.release()
            cv2.destroyAllWindows()
            return None
        
        image_path = f"./{name}_{i+1}.jpg"
        cv2.imwrite(image_path, frame)
        frames.append(frame)
    
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"{num_images} images captured and saved.")
    return frames

In [57]:
import cv2
import time

def capture_images(name, num_images=5, interval=3):
    cap = cv2.VideoCapture(0)
    frames = []
    
    print(f"Preparing to capture {num_images} images. Please look at the camera.")
    
    for i in range(num_images):
        start_time = time.time()
        while True:
            ret, frame = cap.read()
            if not ret:
                print(f"Failed to capture image {i+1}.")
                cap.release()
                cv2.destroyAllWindows()
                return None

            elapsed_time = time.time() - start_time
            remaining_time = interval - elapsed_time

            if remaining_time > 0:
                cv2.putText(frame, f"Capturing in {int(remaining_time) + 1} seconds...",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.imshow('Capture Timer', frame)
                cv2.waitKey(1)
            else:
                image_path = f"./{name}_{i+1}.jpg"
                cv2.imwrite(image_path, frame)
                frames.append(frame)
                print(f"Captured image {i+1}.")
                break
    
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"{num_images} images captured and saved.")
    return frames



In [58]:
def detect_and_crop_faces(frames):
    face_images = []
    for frame in frames:
        try:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            if result:
                face_data = result[0]['region']
                x, y, w, h = face_data['x'], face_data['y'], face_data['w'], face_data['h']
                face_image = frame[y:y+h, x:x+w]
                face_images.append(face_image)
        except Exception as e:
            print(f"An error occurred during face detection: {e}")
    
    return face_images

In [59]:
def calculate_mean_encoding(face_images):
    encodings = []
    for face_image in face_images:
        try:
            encoding = DeepFace.represent(face_image, model_name='VGG-Face', enforce_detection=False)[0]
            encodings.append(encoding["embedding"])
        except Exception as e:
            print(f"An error occurred during encoding: {e}")
    
    if encodings:
        mean_encoding = np.mean(encodings, axis=0)
        return mean_encoding
    return None

In [60]:
def insert_user():
    name = input("Enter the user's name: ")
    frames = capture_images(name)
    
    if frames is None:
        return
    
    face_images = detect_and_crop_faces(frames)
    
    if not face_images:
        print("No face detected in the images. Please try again.")
        for i in range(len(frames)):
            os.remove(f"./{name}_{i+1}.jpg")
        return
    mean_encoding = calculate_mean_encoding(face_images)
    
    if mean_encoding is None:
        print("Failed to encode faces. Please try again.")
        for i in range(len(frames)):
            os.remove(f"./{name}_{i+1}.jpg")
        return
    
    try:
        with sqlite3.connect('emp.db') as conn:
            cursor = conn.cursor()
            cursor.execute('INSERT INTO Users (Name, FaceEncoding) VALUES (?, ?)', (name, pickle.dumps(mean_encoding)))
            conn.commit()
            print(f"User {name} added successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        for i in range(len(frames)):
            # os.remove(f"./{name}_{i+1}.jpg")
            print(f"Temporary image file {name}_{i+1}.jpg deleted.")
        print("All temporary files deleted.")

insert_user()


Preparing to capture 5 images. Please look at the camera.
Captured image 1.
Captured image 2.
Captured image 3.
Captured image 4.
Captured image 5.
5 images captured and saved.
An error occurred during face detection: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
An error occurred during face detection: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
An error occurred during face detection: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
User rohit added successfully.
Temporary image file rohit_1.jpg deleted.
Temporary image file rohit_2.jpg deleted.
Temporary image file rohit_3.jpg deleted.
Temporary image file rohit_4.jpg deleted.
Temporary image file rohit_5.jpg deleted.
All temporary files deleted.


In [61]:
def load_users():
    users = {}
    try:
        with sqlite3.connect('emp.db') as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM Users')
            rows = cursor.fetchall()
            for row in rows:
                Id, Name, FaceEncoding = row
                users[Id] = (Name, pickle.loads(FaceEncoding))
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    return users

In [62]:
def recognize_and_mark_attendance(frame, users):
    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        for (x, y, w, h) in faces:
            face_region = frame[y:y+h, x:x+w]
            rgb_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)
            
            result = DeepFace.represent(img_path=rgb_face, model_name='VGG-Face', enforce_detection=False)
            
            if not result or not isinstance(result, list) or not isinstance(result[0], dict):
                print("Failed to retrieve valid embeddings for face.")
                continue
            
            new_embedding = result[0]['embedding']  
            new_embedding = np.array(new_embedding)
            
            recognized = False
            for Id, (name, stored_embedding) in users.items():
                stored_embedding = np.array(stored_embedding)
                distance = np.linalg.norm(new_embedding - stored_embedding)
                print(distance)
                if distance <1.0:  # Adjust threshold as needed
                    print(f"Recognized {name}")
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    recognized = True
                    break
            
            if not recognized:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                
    except Exception as e:
        print(f"An error occurred: {e}")

In [63]:
def main():
    users = load_users()
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        recognize_and_mark_attendance(frame, users)
        
        cv2.imshow('Webcam Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

0.5241788960786032
Recognized Vighnesh
1.2164067926001754
0.8194832409213104
Recognized rohit
0.562238270359747
Recognized Vighnesh
1.1949564701830653
0.7675230522015901
Recognized rohit
0.5670384151343021
Recognized Vighnesh
1.1680165819536572
0.8037055404350778
Recognized rohit
0.5620946448145018
Recognized Vighnesh
1.174238633362404
0.8470723132455067
Recognized rohit
0.7636433482458885
Recognized Vighnesh
1.182996067394699
0.779459973822366
Recognized rohit
0.7336034273421083
Recognized Vighnesh
0.7566878192023347
Recognized Vighnesh
0.6802579806637611
Recognized Vighnesh
0.6612131040904491
Recognized Vighnesh
1.2001260832990965
0.6963451699887541
Recognized rohit
0.7192931682787215
Recognized Vighnesh
0.6494140091143065
Recognized Vighnesh
1.1777410039368459
0.8629625096823845
Recognized rohit
0.620612766759803
Recognized Vighnesh
1.2019220219145037
1.314360145626125
0.6480309288495361
Recognized Vighnesh
0.6202700651260954
Recognized Vighnesh
0.6648556370632662
Recognized Vighnes

In [ ]:
def capture_image(name):
    cap = cv2.VideoCapture(0)
    
    print("Preparing to capture image. Please look at the camera.")
    time.sleep(5)

    ret, frame = cap.read()
    
    if not ret:
        print("Failed to capture image.")
        cap.release()
        cv2.destroyAllWindows()
        return None, None
    
    image_path = "./"+name + ".jpg"
    cv2.imwrite(image_path, frame)
    
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"Image captured and saved to {image_path}.")
    return image_path, frame

def detect_and_crop_face(image_path, frame):
    try:
        result = DeepFace.analyze(image_path, actions=['emotion'], enforce_detection=True)
        if result:
            face_data = result[0]['region']
            x, y, w, h = face_data['x'], face_data['y'], face_data['w'], face_data['h']
            face_image = frame[y:y+h, x:x+w]
            face_cutout_path = 'face_cutout.jpg'
            cv2.imwrite(face_cutout_path, face_image)
            print(f"Face detected and cutout saved to {face_cutout_path}.")
            return face_cutout_path
    except Exception as e:
        print(f"An error occurred during face detection: {e}")
    return None

def insert_user():
    name = input("Enter the user's name: ")
    image_path, frame = capture_image(name)
    if image_path is None:
        return
    face_cutout_path = detect_and_crop_face(image_path, frame)
    if face_cutout_path is None:
        print("No face detected in the image. Please try again.")
        os.remove(image_path)
        return
    
    try:
        with sqlite3.connect('emp.db') as conn:
            cursor = conn.cursor()
            
            encoding = DeepFace.represent(face_cutout_path, model_name='VGG-Face', enforce_detection=False)[0]
            cursor.execute('INSERT INTO Users (Name, FaceEncoding) VALUES (?, ?)', (name, pickle.dumps(encoding)))
            conn.commit()
            print(f"User {name} added successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"Temporary image file {image_path} deleted.")
        if os.path.exists(face_cutout_path):
            os.remove(face_cutout_path)
            print(f"Face cutout file {face_cutout_path} deleted.")

insert_user()

In [46]:
def load_users():
    users = {}
    try:
        with sqlite3.connect('emp.db') as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM Users')
            rows = cursor.fetchall()
            for row in rows:
                Id, Name, FaceEncoding = row
                users[Id] = (Name, pickle.loads(FaceEncoding))
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    return users

def recognize_and_mark_attendance(frame, users):
    try:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = DeepFace.detectFace(rgb_frame, detector_backend='mtcnn', enforce_detection=False)
        if results is None:
            print("No faces detected.")
            return
        for result in results:
            print(result)
            new_embedding = result['embedding']  # Extract embedding from the result
            face_region = result['region']  # Extract face region
            x, y, w, h = face_region['x'], face_region['y'], face_region['w'], face_region['h']
            
            # Convert new_embedding to a numpy array
            new_embedding = np.array(new_embedding)
            
            recognized = False
            for Id, (name, stored_embedding) in users.items():
                stored_embedding = np.array(stored_embedding['embedding'])
                print(new_embedding)
                print(stored_embedding)
                distance = np.linalg.norm(new_embedding - stored_embedding)
                
                if distance < 1.0:  # Adjust threshold as needed
                    print(f"Recognized {name}")
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    recognized = True
                    break
            
            if not recognized:
                # Draw bounding box and label for unrecognized face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    users = load_users()
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        recognize_and_mark_attendance(frame, users)
        
        # Display the annotated frame
        cv2.imshow('Webcam Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


24-08-05 17:57:40 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
1/1 [==============================] - 0s 37ms/step
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
An error occurred: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
24-08-05 17:57:42 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
1/1 [==============================] - 0s 23ms/step
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 

In [53]:
def load_users():
    users = {}
    try:
        with sqlite3.connect('emp.db') as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM Users')
            rows = cursor.fetchall()
            for row in rows:
                Id, Name, FaceEncoding = row
                # Ensure FaceEncoding is deserialized from bytes
                users[Id] = (Name, pickle.loads(FaceEncoding))
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    return users

def recognize_and_mark_attendance(frame, users):
    try:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Get the embeddings for the current frame
        result = DeepFace.represent(img_path=rgb_frame, model_name='VGG-Face', enforce_detection=False)
        
        if not result or not isinstance(result, list):
            print("Failed to retrieve embeddings.")
            return False
        new_embedding = result[0]["embedding"]
        
        new_embedding = np.array(new_embedding)
        for Id, (name, stored_embedding) in users.items():
            stored_embedding = np.array(stored_embedding["embedding"])
            distance = np.linalg.norm(new_embedding - stored_embedding)
            print(distance)
            if distance < 1.0:
                print(f"Recognized {name}")
                return True
        
        print("Face not recognized")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

def main():
    users = load_users()
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        recognize_and_mark_attendance(frame, users)
        
        cv2.imshow('Webcam Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

1.3232652384561006
1.2330103579613887
1.2600069077120624
0.6765102401285111
Recognized nimesh
1.314276477669293
1.2319964351500032
1.255991111985867
0.5715820542682832
Recognized nimesh
1.3284527160795694
1.2576476722092442
1.265381790528269
0.6833516980310229
Recognized nimesh
1.3228067634765959
1.2318720264357599
1.2583896283707094
0.5903134548820611
Recognized nimesh
1.0195663429700093
1.095718114488874
1.3016673607360978
1.315054053274347
Face not recognized
1.3432298628252146
1.2594811733084885
1.2887568692590432
0.71200385309513
Recognized nimesh
1.0090549383543044
0.9946091007913768
Recognized sharan
1.3369538599846347
1.2537944303906374
1.2902619190769808
0.7221736444145481
Recognized nimesh
0.9757377435778818
Recognized Vighnesh
0.9651748258143288
Recognized Vighnesh
1.3345387410599028
1.2392741356328216
1.2705893130581256
0.5947067122228329
Recognized nimesh
